#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-02-19 19:25:04,441] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



In [3]:
#create a single game instance
env = gym.make("FrozenLake8x8-v0")

#start new game
env.reset();

[2017-02-19 19:25:05,798] Making new env: FrozenLake8x8-v0


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

observations: Discrete(64) n= 64
actions: Discrete(4) n= 4


In [5]:
env.step(2)

(0, 0.0, False, {'prob': 0.3333333333333333})

In [6]:
env.render()

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)


In [7]:
#env.P - вероятности

In [8]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(3)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 1
reward: 0.0
is game over?: False
printing new state:
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)


In [9]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [10]:
env.step(action_to_i['up'])
env.render()

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [11]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0,n_actions,n_states)

In [12]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.24981406  0.25094844  0.24953906  0.24969844]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [13]:
def sample_reward(env, policy, t_max=500):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for _ in range(t_max):
        action = policy[s]
        s,r,done,_=env.step(action)
        total_reward+= r
        if (done):
            break
    return total_reward

In [14]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [15]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [16]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    
    assert ((env.spec.id == "FrozenLake-v0") or (env.spec.id == "FrozenLake8x8-v0")), "this function only works with frozenlake 4x4"
    if env.spec.id == "FrozenLake-v0":
        lake = "SFFFFHFHFFFHHFFG"
    elif env.spec.id == "FrozenLake8x8-v0":
        lake = "SFFFFFFFFFFFFFFFFFFHFFFFFFFFFHFFFFFHFFFFFHHFFFHFFHFFHFHFFFFHFFFG"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow, tile in zip(arrows, lake)]
    
    if env.spec.id == "FrozenLake-v0":
        for i in range(0, 16, 4):
            print(' '.join(signs[i:i+4]))
    elif env.spec.id == "FrozenLake8x8-v0":
        for i in range(0, 64, 8):
            print(' '.join(signs[i:i+8]))

print("random policy:")
print_policy(get_random_policy())

random policy:
> ^ ^ < > > v v
v > ^ ^ > v < >
< v > H < v v v
^ > < < ^ H ^ ^
> v < H v v v ^
> H H > > v H ^
^ H v ^ H ^ H v
^ v > H v > > G


### Random search

In [17]:
import tqdm

In [18]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(1000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")
        print_policy(best_policy)

  0%|          | 5/1000 [00:00<00:54, 18.32it/s]

New best score: 0.0
Best policy:
^ ^ > ^ v ^ v v
< > ^ ^ < v ^ <
v ^ ^ H > < > >
^ > < > v H v ^
> ^ < H < < v <
v H H ^ < < H <
^ H < < H > H ^
> ^ ^ H ^ ^ v G


  2%|▏         | 15/1000 [00:00<00:41, 23.63it/s]

New best score: 0.01
Best policy:
> ^ > ^ > > v v
^ > > < > > v <
> > > H < v < v
< < v ^ ^ H > ^
v ^ ^ H ^ ^ ^ v
< H H < > v H <
< H < ^ H v H <
v < v H ^ > v G


 12%|█▏        | 124/1000 [00:04<00:33, 25.92it/s]

...

 18%|█▊        | 177/1000 [00:07<00:36, 22.34it/s]

New best score: 0.04
Best policy:
> > ^ > < < < v
^ ^ ^ ^ ^ > > v
v ^ v H ^ v < >
> ^ ^ < ^ H > <
< v v H < v > >
v H H ^ > v H <
< H ^ v H > H v
v < > H v v ^ G


 22%|██▏       | 218/1000 [00:08<00:31, 25.09it/s]

New best score: 0.05
Best policy:
> v v < < v > >
> ^ ^ v > > > <
> < v H < v > >
< > v < < H > >
v > v H < > ^ >
> H H < ^ < H >
v H < > H v H <
< < v H v > ^ G


100%|██████████| 1000/1000 [00:40<00:00, 19.81it/s]


# Part II Genetic algorithm

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [19]:
def crossover(policy1, policy2, p=0.5): #для каждого действия с вер р влево, с вер 1-р вправо
    my_choice = np.random.choice(2, len(policy1), p=[p, 1-p])
    return np.choose(my_choice, choices=[policy1, policy2])

In [20]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy,get_random_policy(), p=1-p)
    

In [21]:
import numpy as np
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [22]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [23]:
print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(p) for p in pool]


initializing...


In [24]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [25]:
from random import choice
choice(pool)

array([1, 2, 3, 0, 1, 1, 2, 0, 3, 3, 3, 1, 1, 0, 0, 2, 2, 2, 3, 3, 1, 1, 3,
       1, 2, 1, 1, 1, 3, 2, 0, 1, 0, 3, 1, 1, 2, 0, 2, 3, 0, 2, 1, 3, 3, 2,
       0, 0, 1, 3, 2, 3, 0, 0, 1, 3, 0, 1, 0, 0, 3, 0, 3, 2])

In [71]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    crossovered = [crossover(choice(pool),choice(pool)) for _ in range(n_crossovers)]
    mutated = [mutation(choice(pool)) for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])
    print_policy(pool[-1])

Epoch 0:
best score: 0.12
^ ^ > > > > < v
> ^ < ^ > ^ v v
< v ^ H v > v v
v > < < < H v <
> < < H > ^ v >
< H H < < > H >
< H v v H > H >
^ v ^ H < ^ ^ G
Epoch 1:
best score: 0.1
^ ^ > > > > < v
> ^ < ^ > ^ v v
< v < H v > v v
v > < v < H v <
> < < H > ^ v >
< H H < < v H >
^ H > v H > H >
^ ^ ^ H < ^ ^ G
Epoch 2:
best score: 0.12
^ ^ > > > > < v
> ^ < ^ > ^ v v
< v < H v > v v
v > < v < H v <
> < < H > ^ v >
< H H < < v H >
^ H > v H > H >
^ ^ ^ H < ^ ^ G
Epoch 3:
best score: 0.14
> ^ > > > > v <
< ^ v > ^ ^ > v
^ < < H v > v v
^ > < v ^ H > <
> < < H < v v >
> H H ^ < v H >
^ H > ^ H > H >
> ^ ^ H < ^ ^ G
Epoch 4:
best score: 0.12
> ^ > > > > v <
< ^ v > ^ ^ > v
^ < < H v > v v
^ > < v ^ H > <
> < < H < v v >
> H H ^ < v H >
^ H > ^ H > H >
> ^ ^ H < ^ ^ G
Epoch 5:
best score: 0.17
^ ^ > > > > < v
^ v < ^ > ^ ^ v
^ v < H v > v v
^ > < > < H > <
> < < H < v ^ >
> H H < < v H >
< H < ^ H v H >
^ ^ < H < ^ ^ G
Epoch 6:
best score: 0.36
^ ^ > > > > v ^
> ^ > ^ ^ ^ > v
^ < < H v > > >
v >

Значений с семинара уже хватает для того, чтобы стабильно бить 0.7. (Этот код запущен еще до того, как я поменял t_max выше). Попробуем чуть улучшить.

## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover for higher-scorers?
* try to select a more diverse pool, not just best scorers?
* Just tune the f*cking probabilities.

See which combination works best!

Не всегда хватает шагов при хорошей стратегии. Увеличил t_max до 500

In [94]:
def test_length(env, policy, t_max=999999):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for length in range(t_max):
        action = policy[s]
        s,r,done,_=env.step(action)
        total_reward+= r
        if (done):
            break
    print ('length = ', length)
    env.render()
    return total_reward, length

test_length(env, pool[-1])
test_length(env, pool[-1])
test_length(env, pool[-1])
test_length(env, pool[-2])
test_length(env, pool[-2])
test_length(env, pool[-2])
test_length(env, pool[0])
test_length(env, pool[0])
test_length(env, pool[0])
test_length(env, pool[1])
test_length(env, pool[1])
test_length(env, pool[1])




length =  104
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  69
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  78
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  91
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Left)
length =  97
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  55
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  162
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  60
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  62
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  67
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
length =  96
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF


(1.0, 291)

In [26]:

n_epochs = 100 #how many cycles to make
pool_size = 50 #how many policies to maintain
n_crossovers = 25 #how many crossovers to make on each step
n_mutations = 25 #how many mutations to make on each tick

print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(p) for p in pool]

assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

initializing...


In [27]:
choice_prob = np.log2(np.arange(pool_size) + 2)
choice_prob /= np.sum(choice_prob)
print(choice_prob)
indeces = np.arange(pool_size, dtype=int)

[ 0.00454792  0.00720829  0.00909585  0.01055995  0.01175621  0.01276764
  0.01364377  0.01441658  0.01510788  0.01573323  0.01630414  0.01682932
  0.01731556  0.01776824  0.01819169  0.01858947  0.0189645   0.01931925
  0.0196558   0.01997592  0.02028115  0.02057281  0.02085206  0.0211199
  0.02137724  0.02162487  0.02186348  0.02209373  0.02231616  0.02253131
  0.02273962  0.02294152  0.02313739  0.02332759  0.02351242  0.0236922
  0.02386717  0.02403761  0.02420372  0.02436574  0.02452385  0.02467824
  0.02482908  0.02497653  0.02512074  0.02526185  0.02539998  0.02553527
  0.02566783  0.02579776]


In [119]:
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    
    crossovered = [crossover(pool[np.random.choice(indeces, p=choice_prob)], 
                             pool[np.random.choice(indeces, p=choice_prob)]) for _ in range(n_crossovers)]
    
    mutated = [mutation(choice(pool), 0.03) for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])
    print_policy(pool[-1])

Epoch 0:
best score: 0.06
< < v v > v > >
v ^ ^ > ^ ^ v v
> < > H > ^ v <
^ ^ ^ ^ v H > >
> > > H < v ^ v
< H H ^ v > H v
v H v < H > H >
> ^ < H v > ^ G
Epoch 1:
best score: 0.06
< < v v > v > >
v ^ ^ > ^ ^ v v
> < > H > ^ v <
^ ^ ^ ^ v H > >
> > > H < v ^ v
< H H ^ v > H v
v H v < H > H >
> ^ < H v > ^ G
Epoch 2:
best score: 0.09
^ > v < > > < v
v ^ v ^ v < < ^
< < < H v ^ > v
^ > < ^ > H > >
< > ^ H ^ < < >
^ H H < < < H >
^ H > v H ^ H >
< v ^ H ^ < < G
Epoch 3:
best score: 0.09
^ < v v > > > >
v v ^ ^ v > > v
v < > H > v > <
> v ^ ^ v H > <
v > < H > > ^ v
< H H v > < H >
^ H v > H > H <
v v < H > v ^ G
Epoch 4:
best score: 0.13
^ ^ > > < > > >
v < v ^ ^ v > ^
^ < < H v ^ > ^
^ ^ < v v H > <
< < < H > v > >
> H H > > ^ H <
^ H > ^ H ^ H >
> ^ < H > v v G
Epoch 5:
best score: 0.15
^ ^ > > < > > >
v < v ^ ^ v > ^
^ < < H v ^ > ^
^ ^ < v v H > <
< < < H > v > >
> H H > > ^ H <
^ H > ^ H ^ H >
> ^ < H > v v G
Epoch 6:
best score: 0.27
^ > > > < > > >
v ^ v ^ ^ v > ^
< < < H v ^ > v
^ 

Уменьшил вероятность мутации, увеличил вероятность кроссовера для более клевых, на 40й эпохе сошлось к единице.

In [121]:
best_strategy = pool[-1]
print(best_strategy)

[3 1 2 1 2 1 3 2 3 3 3 3 3 3 3 2 2 0 3 2 3 3 2 2 2 1 3 1 2 0 2 2 2 2 3 2 3
 1 3 2 2 3 1 1 2 2 1 2 3 2 2 1 2 3 2 2 1 3 0 0 3 0 3 1]


# Part III

The frozenlake problem above is just too simple: you can beat it even with a random policy search. Go solve something more complicated.

* __FrozenLake8x8-v0__ - frozenlake big brother (achieve score >0.7)
* __Taxi-v1__ - essentially a maze where you get score for moving passengers to their destinations. (achieve score >5)


### Some tips:
* Random policy search is worth trying as a sanity check, but in general you should expect the genetic algorithm (or anything you devised in it's place) to fare much better that random.
* While _it's okay to adapt the tabs above to your chosen env_, make sure you didn't hard-code any constants there (e.g. 16 states or 4 actions).
* `print_policy` function was built for the frozenlake-v0 env so it will break on any other env. You could simply ignore it OR write your own visualizer for bonus points.
* in function `sample_reward`, __make sure t_max steps is enough to solve the environment__ even if agent is sometimes acting suboptimally. To estimate that, run several sessions without time limit and measure their length.

### Bonus I:
* Gym envs have a condition for "beating the game". E.g. here's the conditions for [Taxi-v1](https://gym.openai.com/envs/Taxi-v1). 
 * If you managed to do that, it's worth uploading your first solution to gym. See `gym.upload(...)` docs. Allbeit it isn't a strong AI (or is it?), uploading your algorithm would be a good start. (and a +point!)

### Bonus II:
* There are environments with continuous state spaces. In fact, most real world environments have this property. While we will dive into methods designed for that later, right now you already can solve them through binarization.
 * Gym has a basic infinite-state-space env called [CartPole](https://gym.openai.com/envs/CartPole-v0) - please start from this one. Solving something more challenging is great, but make sure your algorithm beats cartpole first. Also kudos for submitting.
 * Main idea: if you have something infinite and you want something discrete, you split it into bins. Like what histogram does.
 * Good choice of discretes is critical!
 * If the dimensionality is too high, you can try to reduce it (PCA/autoencoders)

